In [ ]:
# default_exp core

# doc_collection

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import importlib
import os
import sys
import inspect
import json
import pkg_resources
import pkgutil
import pandas
import warnings
import pydoc
from pathlib import Path

In [ ]:
#hide
def replace(list_, str1, str2):
    i = list_.index(str1)
    return list_[:i] + [str2] + list_[i + 1:]

In [ ]:
#export
def get_top_hundred():
    if not os.path.isfile('data/top.json'):
        os.system('curl -X GET "https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json" > data/top.json')
        
    f = open('data/top.json')
    data = json.load(f)
    
    return [dictionary['project'] for dictionary in data['rows'][:100]]

In [ ]:
get_top_hundred()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  280k  100  280k    0     0   387k      0 --:--:-- --:--:-- --:--:--  387k


['boto3',
 'botocore',
 'urllib3',
 'setuptools',
 'requests',
 's3transfer',
 'six',
 'python-dateutil',
 'certifi',
 'idna',
 'pyyaml',
 'typing-extensions',
 'charset-normalizer',
 'pip',
 'numpy',
 'google-api-core',
 'wheel',
 'cryptography',
 'pyparsing',
 'packaging',
 'jmespath',
 'awscli',
 'rsa',
 'pyasn1',
 'importlib-metadata',
 'zipp',
 'pyjwt',
 'colorama',
 'pytz',
 'click',
 'pandas',
 'protobuf',
 'attrs',
 'cffi',
 'oauthlib',
 'jinja2',
 'requests-oauthlib',
 'markupsafe',
 'pycparser',
 'docutils',
 'google-auth',
 'cachetools',
 'pyasn1-modules',
 'wrapt',
 'googleapis-common-protos',
 'psutil',
 'isodate',
 'pyarrow',
 'sqlalchemy',
 'azure-core',
 'lxml',
 'chardet',
 'tomli',
 'msrest',
 'async-timeout',
 'grpcio',
 'decorator',
 'aiobotocore',
 'werkzeug',
 'pillow',
 'aiohttp',
 'multidict',
 'beautifulsoup4',
 'soupsieve',
 'scipy',
 'yarl',
 'google-cloud-storage',
 'py',
 'fsspec',
 'google-cloud-bigquery',
 'importlib-resources',
 'pytest',
 'greenlet',
 '

In [ ]:
#export 
def pip_top_hundred():
    lib_names = get_top_hundred()
    lib_names = replace(lib_names, 'msrest', 'msrestazure')
    for lib_name in lib_names:
        os.system('pip install ' + lib_name)

In [ ]:
#hide
def val(key):
    return key[0] != '_' and key[-1] != '_'

In [ ]:
#hide
def extr(module_name):
    names = []
    texts = []
    paths = []
    
    finded_names, finded_texts, finded_paths = recclass(module_name, module_name.__name__)
    names.extend(finded_names)
    texts.extend(finded_texts)
    paths.extend(finded_paths)
            
    if hasattr(module_name, '__path__'):
        file = module_name.__path__
    else:
        file = [inspect.getfile(module_name)]
        
    for importer, key, ispkg in pkgutil.iter_modules(file): 
        if val(key) and ispkg and hasattr(module_name, key):
            if key != module_name.__name__ and fullname(getattr(module_name, key)).startswith(fullname(module_name)):
                finded_names, finded_texts, finded_paths = extr(getattr(module_name, key))
                names.extend(finded_names)
                texts.extend(finded_texts)
                paths.extend(finded_paths)
                
    return names, texts, paths

In [ ]:
#hide 
def recclass(cl, path):
    names = []
    texts = []
    paths = []
    
    names.append(cl)
    texts.append(pydoc.render_doc(cl, renderer=pydoc.TextDoc()))
    paths.append(path)
    
    for key_, value_ in inspect.getmembers(cl, inspect.isroutine):
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)
    
    for key_, value_ in inspect.getmembers(cl, inspect.isfunction):
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)
               
    for key_, value_ in inspect.getmembers(cl, inspect.isdatadescriptor): 
        if val(key_) and hasattr(cl, key_):
            names.append(getattr(cl, key_))
            texts.append(pydoc.render_doc(getattr(cl, key_), renderer=pydoc.TextDoc()))
            paths.append(path + '.' + key_)
             
    for key_, value_ in inspect.getmembers(cl, inspect.isclass): 
        if val(key_) and hasattr(cl, key_):
            if fullname(getattr(cl, key_)).startswith(fullname(cl)):
                finded_names, finded_texts, finded_paths = recclass(getattr(cl, key_), path + '.' + key_)
                names.extend(finded_names)
                texts.extend(finded_texts)
                paths.extend(finded_paths)
                
    return names, texts, paths


In [ ]:
#hide
def fullname(o):
    
    if inspect.ismodule(o):
        return o.__name__
    
    if inspect.isfunction(o) or inspect.isclass(o) or inspect.ismethod(o):
        return o.__module__ + '.' + o.__name__
    
    if inspect.isroutine(o):
        if hasattr(o, '__name__'):
            return o.__class__.__module__ + '.' + o.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__
        
    if inspect.isdatadescriptor(o):
        if o.__class__.__name__ == 'property' and o.fget != None:
            if o.fget.__module__ != None:
                return o.fget.__module__ + '.' + o.fget.__name__ 
            else:
                return o.fget.__name__
        else:
            return o.__class__.__module__ + '.' + o.__class__.__name__
        
    return o.__class__.__name__

In [ ]:
#export
def extract():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning) 
    
    installed_packages = pkg_resources.working_set
    installed_packages_list = sorted(["%s" % i.key for i in installed_packages])
    
    installed_packages_list.remove('transformers')
    
    unsolved_package_names = ['beautifulsoup4', 'typing-extensions', 'pyyaml', 'argon2-cffi', 'jupyter-client', 'jupyter-core']
    unsolved_import_names = ['bs4', 'typing', 'yaml', 'argon2', 'jupyter_client', 'jupyter_core']
    
    for i in range(len(unsolved_package_names)):
        installed_packages_list = replace(installed_packages_list, unsolved_package_names[i], unsolved_import_names[i])
        
    names = []
    texts = []
    paths = []
    
    c = 0
    for lib_name in installed_packages_list:
        try:
            importlib.import_module(lib_name)
        
            finded_names, finded_texts, finded_paths = extr(sys.modules[lib_name])
            names.extend(finded_names)
            texts.extend(finded_texts)
            paths.extend(finded_paths)
            
            print(len(names) - c, 'files from', lib_name, 'added')
            c = len(names)
            
        except Exception:
            try:
                metadata_dir = pkg_resources.get_distribution(lib_name).egg_info
                for name in open('%s/%s' % (metadata_dir, 'top_level.txt')).read().rstrip().split('\n'):
                    if name != '' and val(name):
                        importlib.import_module(name)
            
                        finded_names, finded_texts, finded_paths = extr(sys.modules[name])
                        names.extend(finded_names)
                        texts.extend(finded_texts)
                        paths.extend(finded_paths)
            
                        print(len(names) - c, 'files from', name, 'added')
                        c = len(names)
                    
            except Exception:
                print('--------------- error during ' + lib_name + ' documentation extracting')
                
    doc = pandas.DataFrame({'text': texts, 'path': paths})
    doc_wo_dupl = doc.drop_duplicates(subset=['text'])
    d = doc_wo_dupl.groupby('text').agg(paths = ('path', lambda x: list(x)), library = ('path', lambda x: list(x)[0].split('.')[0])).reset_index()
    return d

In [ ]:
extract()

1 files from absl added
41 files from adal added
1 files from aiobotocore added
709 files from aiohttp added
59 files from aioitertools added
15 files from aiosignal added
33 files from argon2 added
1 files from asgiref added
3 files from asn1crypto added
35 files from asttokens added
25 files from astunparse added
12 files from async_generator added
16 files from async_timeout added
29 files from attrs added
1 files from awscli added
1 files from azure added
1 files from azure added
1 files from azure added
3 files from backcall added
20 files from backports.zoneinfo added
2215 files from bs4 added
35 files from beir added
11 files from bleach added
85 files from boto3 added
117 files from botocore added
198 files from cachetools added
5 files from certifi added
90 files from cffi added
16 files from chardet added
124 files from charset_normalizer added
628 files from click added
30 files from colorama added
417 files from cryptography added
18 files from cycler added
1491 files from 

2022-07-27 15:36:29.319963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-27 15:36:29.320250: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


131 files from eli5 added
83 files from entrypoints added
2 files from et_xmlfile added
1 files from execnb added
30 files from executing added
8255 files from faiss added
1 files from fastcore added
290 files from fastjsonschema added
50 files from filelock added
473 files from flask added
146 files from flatbuffers added
4 files from fontTools added
49 files from frozenlist added
190 files from fsspec added
1 files from future added
126 files from gast added
4 files from google added
4 files from google added
3 files from google_auth_oauthlib added
48 files from google added
48 files from google added
48 files from google added
48 files from google added
14 files from google_crc32c added
7 files from pasta added
48 files from google added
48 files from google added
197 files from graphviz added
21 files from greenlet added
522 files from grpc added
1 files from grpc_status added
77 files from h11 added
236 files from h5py added
251 files from huggingface_hub added
43 files from idna 

/home/satoru/projects/text-retrieval/my_project_env/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


187 files from prometheus_client added
1332 files from prompt_toolkit added
107 files from proto added
48 files from google added
243 files from psutil added
80 files from psycopg2 added
92 files from ptyprocess added
16 files from pure_eval added
1 files from py added
2843 files from pyarrow added
1 files from pyasn1 added
1 files from pyasn1_modules added
343 files from pycparser added
136 files from pygments added
97 files from jwt added
1 files from OpenSSL added
6736 files from pyparsing added
337 files from pyrsistent added
147 files from socks added
64 files from sockshandler added
71 files from pytest added
145 files from dateutil added
10 files from pytrec_eval added
2 files from pytrec_eval_ext added
71 files from pytz added
2789 files from yaml added
506 files from zmq added
1 files from qtconsole added
12 files from qtpy added
42 files from regex added
192 files from requests added
114 files from requests_oauthlib added
126 files from responses added
61 files from rsa added

/home/satoru/projects/text-retrieval/my_project_env/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


66157 files from torch added
7591 files from torchvision added
2 files from tornado added
165 files from tqdm added
1894 files from traitlets added
370 files from trio added
70 files from trio_websocket added
158 files from typing added
231 files from urllib3 added
6 files from wcwidth added
18 files from webencodings added
155 files from websocket added
330 files from werkzeug added
1 files from wheel added
2 files from widgetsnbextension added
36 files from wrapt added
33 files from wsproto added
76 files from xxhash added
66 files from yarl added
86 files from zipp added


,text,paths,library
0,Python Library Documentation: Any in module tr...,[ipykernel.comm.Comm.log],ipykernel
1,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.columns],pandas
2,Python Library Documentation: AxisProperty\n\n...,[pandas.Series.index],pandas
3,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.index],pandas
4,Python Library Documentation: Bool in module o...,[openpyxl.chart.AreaChart.roundedCorners],openpyxl
...,...,...,...
30103,Python Library Documentation: property\n\n ...,[faiss.BufferList.wp],faiss
30104,Python Library Documentation: reify\n\n,[aiohttp.ClientResponse.content_disposition],aiohttp
30105,Python Library Documentation: reify\n\n A s...,[aiohttp.ClientResponse.history],aiohttp
30106,Python Library Documentation: reify\n\n Ret...,[aiohttp.BodyPartReader.filename],aiohttp
